# Проект для «киВшоп»

## Описание проекта

Интернет-магазин «Твиишоп» запускает новый сервис. Добавили функцию редактирования и дополнения описания товаров. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужна модель, которая будет определять токсичные комментарии и отправлять их на модерацию.

## Описание данных

*Для повторения проекта, можно использовать открытый датасет с kaggle: https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data*

- `text` - текст комментария
- `toxic` - целевой признак (1 - токсичный комментарий, 0 - нет)

## Подготовка

In [44]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Embedding, LSTM
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import keras.backend as K

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline, FunctionTransformer

from transformers import DistilBertModel, DistilBertTokenizer
from transformers import  AutoTokenizer, AutoModel

import torch

from lightgbm import LGBMClassifier

from tqdm import notebook

import joblib

In [2]:
try:
    data = pd.read_csv('toxic_comments.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

Создадим подвыборку из n строк, чтобы ускорить подбор кодировки и выбор модели

In [3]:
# make small sample
# data = data.sample(n=100, random_state=42)

Посмотрим на данные.

In [4]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Удалим лишний столбец с индексами.

In [5]:
# drop Unnamed: 0 column
data.drop('Unnamed: 0', axis=1, inplace=True)

Проверим сколько строк и проверим на пропуски.

In [6]:
data.info()

Посмотрим на баланс классов.

In [7]:
data['toxic'].value_counts()

toxic
0    143106
1     16186
Name: count, dtype: int64

Наблюдается сильный дисбаланс классов. При обучении моделей, нужно будет это учитывать.

Инициализируем токенизатор и модель для кодировки текста. Будем использовать unitary/toxic-bert (основанная на DistilBert), так как он специально обучен для задачи классификации токсичности комментариев.

In [8]:
model_name = "unitary/toxic-bert"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Токенизируем текст и записываем в переменную tokenized. Выводим размер токенизированного текста на экран.

In [9]:
tokenized = data['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True)))
tokenized.shape

(159292,)

Создаем векторы одинаковой длины, заполняя недостающие значения нулями. И создаем маску векторов.

In [10]:
padded = np.array([i + [0]*(512-len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

Производим создание эмбеддингов, с помощью предобученной модели.

In [11]:
batch_size = 250
embeddings = []

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

for i in notebook.tqdm(range(padded.shape[0] // batch_size + 1)):
    batch = torch.tensor(padded[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.tensor(attention_mask[batch_size*i:batch_size*(i+1)])

    with torch.no_grad():
        batch_embeddings = model(batch.to(device), attention_mask=attention_mask_batch.to(device))

    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

  0%|          | 0/638 [00:00<?, ?it/s]

Преобразуем список в двумерную матрицу. И сохраняем в переменную embeddings.

In [12]:
embeddings = np.concatenate(embeddings)

Создаем ячейку для сохранения эмбеддингов в файл.

In [13]:
# save embeddings
# embeddings_df = pd.DataFrame(embeddings)
# embeddings_df.to_parquet('embeddings_toxic-comment-model.parquet')

Создаем ячейку для загрузки эмбеддингов из файла.

In [14]:
# load embeddings
# embeddings = pd.read_parquet('embeddings_distilBert.parquet')

Проверим размерность эмбеддингов.

In [15]:
embeddings.shape

(159292, 768)

Разделим данные на обучающую, валидационную и тестовую выборки. Соотношение 75/25/10.

In [16]:
# split data on train, valid and test
X_train_valid, X_test, y_train_valid, y_test = train_test_split(embeddings, data['toxic'], test_size=0.1, random_state=42, stratify=data['toxic'])

In [17]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.3, random_state=42, stratify=y_train_valid)

Проверим размерность выборок.

In [18]:
(X_train_valid.shape, y_train_valid.shape), (X_train.shape, y_train.shape), (X_valid.shape, y_valid.shape), (X_test.shape, y_test.shape)

(((143362, 768), (143362,)),
 ((100353, 768), (100353,)),
 ((43009, 768), (43009,)),
 ((15930, 768), (15930,)))

## Обучение

Обучим модель LogisticRegression с проверкой threshold.

In [19]:
# train logistic regression
lr = LogisticRegression(random_state=42, solver='liblinear')
lr.fit(X_train, y_train)

# Obtain predicted probabilities on the test set
probas = lr.predict_proba(X_valid)[:, 1]

# Define a range of threshold values
thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Evaluate the model performance for each threshold value
for threshold in thresholds:
    y_pred = (probas >= threshold).astype(int)
    precision = precision_score(y_valid, y_pred)
    recall = recall_score(y_valid, y_pred)
    f1 = f1_score(y_valid, y_pred)
    print(f"Threshold: {threshold:.1f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}")

- Threshold: 0.1, Precision: 0.86, Recall: 0.98, F1-score: 0.92
- Threshold: 0.2, Precision: 0.90, Recall: 0.98, F1-score: 0.94
- Threshold: 0.3, Precision: 0.92, Recall: 0.97, F1-score: 0.94
- Threshold: 0.4, Precision: 0.94, Recall: 0.96, F1-score: 0.95
- Threshold: 0.5, Precision: 0.95, Recall: 0.95, F1-score: 0.95
- Threshold: 0.6, Precision: 0.96, Recall: 0.93, F1-score: 0.94
- Threshold: 0.7, Precision: 0.97, Recall: 0.91, F1-score: 0.94
- Threshold: 0.8, Precision: 0.98, Recall: 0.89, F1-score: 0.93
- Threshold: 0.9, Precision: 0.99, Recall: 0.84, F1-score: 0.91

Как мы можем видеть, при threshold = 0.5, мы получаем наилучший результат. Значит, не стоит менять threshold и делать баланс классов в других моделях.

Обучаем модель LightGBM с подбором гиперпараметров.

In [35]:
# train lightgbm
lgbm = LGBMClassifier(random_state=42)

param_grid = {
    'n_estimators': [100, 150, 200],
    'max_depth': [8, 10 , 12],
    'learning_rate': [0.05, 0.1, 0.15]
}

grid_search = RandomizedSearchCV(lgbm, param_grid, cv=2, scoring='f1', random_state=42, n_jobs=-1)
grid_search.fit(X_train_valid, y_train_valid)

print(grid_search.best_params_)
print(f'F1-score for LightGBM: {grid_search.best_score_:.2f}')

{'n_estimators': 100, 'max_depth': 10, 'learning_rate': 0.1}

F1-score for LightGBM: 0.94

In [36]:
lgbm = LGBMClassifier(random_state=42, **grid_search.best_params_)
lgbm.fit(X_train, y_train)

y_pred = lgbm.predict(X_valid)
print(f'F1-score for LightGBM: {f1_score(y_valid, y_pred):.2f}')

F1-score for LightGBM: 0.94


In [21]:
def get_f1(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val


In [31]:
nn = Sequential()
nn.add(Dense(128, activation='relu', input_shape=(768,)))
nn.add(Dropout(0.2))
nn.add(Dense(128, activation='relu'))
nn.add(Dropout(0.2))
nn.add(Dense(64, activation='relu'))
nn.add(Dense(1, activation='sigmoid'))

nn.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=[get_f1])

nn.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=150,
    validation_data=(X_valid, y_valid),
    verbose=1,
    callbacks=[EarlyStopping(monitor='val_f1_score', mode='max', patience=5, restore_best_weights=True)]
)

F1-score for Neural Network: 0.93

## Анализ моделей

Все модели показали хороший результат. Лучшей моделью оказалась LogisticRegression. Нужно проверить не переобучилась ли она на тренировочных данных. И для сравнения проверим все остальные модели. Также, сравним результаты с константной моделью.

In [42]:
# test constant model
dummy = DummyClassifier(strategy='constant', constant=1)
dummy.fit(X_train, y_train)
print(f'F1 score for DummyClassifier: {f1_score(y_test, dummy.predict(X_test))}')


F1 score for DummyClassifier: 0.18451193800216537


In [27]:
# test logistic regression
print(f'F1 score for LogisticRegression: {f1_score(y_test, lr.predict(X_test))}')



F1 score for LogisticRegression: 0.9499539453484802


In [37]:
# test lightgbm
print(f'F1 score for LGBMClassifier: {f1_score(y_test, lgbm.predict(X_test))}')



F1 score for LGBMClassifier: 0.9474976972674239


In [32]:
# test neural network
print(f'F1 score for Neural Network: {nn.evaluate(X_test, y_test)[1]}')


498/498 [==============================] - 1s 2ms/step - loss: 0.0288 - get_f1: 0.9252


F1 score for Neural Network: 0.9251944422721863


Все модели имеют высокий показатель F1-меры и он почти равен на вариационных данных. Также, по сравнению с константным предсказанием наши модели показывают отличный результат. Из этого можно сделать вывод, что модели не переобучились и дают адекватные предсказания. Обучим LogisticRegression на всех данных и снова проверим ее на тестовых данных.

In [39]:
lr = LogisticRegression(random_state=42, solver='liblinear')
lr.fit(X_train_valid, y_train_valid);

In [40]:
# test logistic regression with train and valid data
print(f'F1 score for LogisticRegression: {f1_score(y_test, lr.predict(X_test))}')



F1 score for LogisticRegression: 0.9504767763764994


## Выводы

В ходе работы были выполнены следующие шаги:

- Данные были загружены и подготовлены для обучения моделей.
- Были обучены модели LogisticRegression, LGBMClassifier, Neural Network.
- Были проведены тесты моделей на тестовых данных.
- Были сделаны выводы по результатам тестов.

В результате работы была достигнута F1-мера не менее 0.75. А если быть точным то 0.95. Также, было обнаружено, что модели не переобучились и дают адекватные предсказания. Адекватность модели проверялась сравнением с константной моделью. Также, было обнаружено, что модель LogisticRegression показала лучший результат. Поэтому, она была обучена на всех данных и проверена на тестовых данных. И она показала отличный результат. F1-мера на тестовых данных составила 0.95. Поэтому можно сделать вывод, что модель LogisticRegression может быть использована для предсказания токсичности комментариев.